In [ ]:
import os
import geopandas as gpd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")
local_crs = 32636
# path to data
example_data_path = "./data"

In [ ]:
!pip install --upgrade pip -q
!pip install osm2geojson -q

In [ ]:
from nirma_masterplans_data_getter_whole_city import DataGetter as dg

dgetter = dg(city_crs=32636)

city_geometry = dgetter.get_city_geometry('Санкт-Петербург', city_admin_level=5)
# water_geometry = dgetter.get_water_geometry(city_name='Санкт-Петербург', water_buffer=5)
# roads_geometry = dgetter.get_roads_geometry(city_geometry=city_geometry, roads_buffer=5)
# railways_geometry = dgetter.get_railways_geometry(city_name='Санкт-Петербург', railways_buffer=5)

## Input data fetch and parameters setting

In [ ]:
from masterplan_tools.method.blocks import CutParameters

# city_geometry = gpd.read_parquet(os.path.join(example_data_path, "city_geometry.parquet")).to_crs(local_crs)
# water_geometry = gpd.read_parquet(os.path.join(example_data_path, "water_geometry.parquet")).to_crs(local_crs)
# roads_geometry = gpd.read_parquet(os.path.join(example_data_path, "roads_geometry.parquet")).to_crs(local_crs)
# railways_geometry = gpd.read_parquet(os.path.join(example_data_path, "railways_geometry.parquet")).to_crs(local_crs)

#basically we need just these 4 geometries to cut some blocks
# cut_params = CutParameters(
#   city=city_geometry,
#   water=water_geometry,
#   roads=roads_geometry,
#   railways=railways_geometry
# )

In [ ]:
blocks = gpd.read_parquet('blocks_general.parquet')

In [ ]:
all(blocks.geometry.geom_type.isin(['Polygon']))

To improve our method we should use land use filtering. If we don't set landuse parameters, no LU filtering will be applied to the blocks

In [ ]:
from masterplan_tools.method.blocks import LandUseParameters

no_development = gpd.read_file(os.path.join(example_data_path, "no_development_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
# no_development = no_development[no_development['RAYON']=='Василеостровский']
landuse = gpd.read_file(os.path.join(example_data_path, "landuse_zone_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
buildings_geom = gpd.read_file(os.path.join(example_data_path, "buildings_blocks.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)



In [ ]:
osm_landuse = gpd.read_parquet('osm_landuse.parquet')

In [ ]:
city_geometry8 = dgetter.get_city_geometry('Санкт-Петербург', city_admin_level=5, dissolve=False)

In [ ]:
city_geometry8.explore()

In [ ]:
city_geometry8 = city_geometry8.explode(ignore_index=True).reset_index(drop=False)

In [ ]:
city_geometry8.head()

In [ ]:
city_geometry8 = city_geometry8[city_geometry8['id']==1114252]

In [ ]:
from tqdm import tqdm
import pandas as pd
from masterplan_tools.method.blocks.landuse_filter import LuFilter

res = gpd.GeoDataFrame()
for i in tqdm(range(city_geometry8.shape[0])):
    mun_geom = gpd.GeoDataFrame(geometry=[city_geometry8.iloc[i, 2]], crs=city_geometry8.crs.to_epsg())
    osm_landuse_mun = osm_landuse.clip(mun_geom)
    no_development_mun = no_development.clip(mun_geom)
    landuse_mun = landuse.clip(mun_geom)
    buildings_geom_mun = buildings_geom.clip(mun_geom)
    blocks_mun = blocks.clip(mun_geom)

    lu_params = LandUseParameters(
        no_development=no_development_mun,
        landuse=landuse_mun,
        buildings=buildings_geom_mun
        )
    
    res = pd.concat([res, LuFilter(blocks_mun, landuse_geometries=lu_params).filter_lu(osm_landuse_mun)], axis=0)

In [ ]:
res = res[~res.is_empty]
res = res.reset_index().drop_duplicates(subset='index')

In [ ]:
res.to_file('res.geojson')

In [ ]:
res.reset_index().drop_duplicates(subset='index').explore()

In [ ]:
res.explore()

In [ ]:
from masterplan_tools.method.blocks.landuse_filter import LuFilter

res = LuFilter(blocks, landuse_geometries=lu_params).filter_lu(osm_landuse)

In [ ]:
no_development[no_development.geometry.geom_type.isin(['Polygon, MultiPolygon'])]

In [ ]:
no_development[no_development.geom_type.isin(['Polygon', 'MultiPolygon'])]


In [ ]:
no_development.geometry.geom_type.isin(['MultiPolygon'])

In [ ]:
no_development = no_development.loc[:, ['OBJECT_ID', 'geometry']]
no_development[no_development.geometry.geom_type.isin(['Polygon, MultiPolygon'])]

In [ ]:
res.to_file('res1.geojson')

In [ ]:
res.explore(legend='landuse')

In [ ]:
res.landuse.unique()

To generate city blocks GeoDataFrame we use the `BlockCutter` class

In [ ]:
from masterplan_tools.method import BlocksCutter

blocks = BlocksCutter(
  cut_parameters=cut_params, 
  lu_parameters=lu_params,
).get_blocks()

In [ ]:
''' 
5 минут на оверлэи дорог
3 минуты на скачать лэндюз

'''

In [ ]:
blocks.to_gdf().head()

There are three landuse tags in the blocks gdf:
1. 'no_dev_area' -- according to th no_debelopment gdf and cutoff without any buildings or specified / selected landuse types;
2. 'selected_area' -- according to the landuse gdf. We separate theese polygons since they have specified landuse types;
3. 'buildings' -- there are polygons that have buildings landuse type. 

In further calculations we will use the in the following steps:
- Only 'buildings' -- to find clusters of buildings in big polygons;
- All of them while calculating the accessibility times among city blocks;
- All of them except 'no_dev_area' while optimizing the development of new facilities.

In [ ]:
blocks.to_gdf().plot(column='landuse', legend=True)

Save the result for the next example

In [ ]:
blocks.to_gdf().to_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet"))